# JPA Planck Spectroscopy

This notebook contains the code for running the Planck Spectroscopy experiment on the JPA to obtain its noise-temperature graph using an automated temperature ramp-up.

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import h5py
import inspect
from tqdm import tqdm
import sys
import math
import presto
from presto import lockin, utils, hardware
from presto.hardware import AdcFSample, AdcMode, DacFSample, DacMode
from blueftc.BlueforsController import BlueFTController

# Reload credentials module to get latest changes
import importlib
import JPA_credentials
importlib.reload(JPA_credentials)
from JPA_credentials import API_KEY, IP_ADDRESS, PORT_NUMBER, MXC_ID, HEATER_ID, PID_CALIB_FILE

# Timeout for hardware communication
import requests

_real_get = requests.get
def get_with_timeout(*args, **kwargs):
    kwargs.setdefault("timeout", (3.0, 5.0))  # (connect_timeout, read_timeout)
    return _real_get(*args, **kwargs)

requests.get = get_with_timeout

## `READ`Commands

- Reading the temperature of each channel (1 to 8)
- Reading the resistance of each channel (1 to 8)
- Reading the status of the mixing chamber heater

In [17]:
controller = BlueFTController(
	ip=IP_ADDRESS, 
	port=PORT_NUMBER, 
	key=API_KEY, 
	mixing_chamber_channel_id=MXC_ID, 
	mixing_chamber_heater_id=HEATER_ID, 
	pid_calib_path=PID_CALIB_FILE
	)

active_channels = [7]

for ch in active_channels:
	print(f"Channel {ch} temp: {controller.get_channel_temperature(ch)} Kelvin")
	print(f"Channel {ch} resistance: {controller.get_channel_resistance(ch)} Ohm")

print(f"MXC heater status: {controller.get_mxc_heater_status()}")
print(f"MXC heater power: {controller.get_mxc_heater_power()} uW")
print(f"MXC heater PID: {controller.get_mxc_heater_mode()}")
print(f"MXC heater setpoint: {controller.get_mxc_heater_setpoint()} K")
print(f"MXC heater PID config: {controller.get_mxc_heater_pid_config()}")

2026-02-02 14:09:50,313 :-- INFO --: Requesting value: temperature  from channel 7
2026-02-02 14:09:50,313 :-- INFO --: Requesting value: temperature  from channel 7
2026-02-02 14:09:50,313 :-- INFO --: Requesting value: temperature  from channel 7
2026-02-02 14:09:50,313 :-- INFO --: Requesting value: temperature  from channel 7
2026-02-02 14:09:50,313 :-- INFO --: Requesting value: temperature  from channel 7
2026-02-02 14:09:50,313 :-- INFO --: Requesting value: temperature  from channel 7
2026-02-02 14:09:50,313 :-- INFO --: Requesting value: temperature  from channel 7
2026-02-02 14:09:50,313 :-- INFO --: Requesting value: temperature  from channel 7
2026-02-02 14:09:50,313 :-- INFO --: Requesting value: temperature  from channel 7
2026-02-02 14:09:50,313 :-- INFO --: Requesting value: temperature  from channel 7
2026-02-02 14:09:50,313 :-- INFO --: Requesting value: temperature  from channel 7
2026-02-02 14:09:50,313 :-- INFO --: Requesting value: temperature  from channel 7
2026

Channel 7 temp: 0.0 Kelvin


2026-02-02 14:09:56,381 :-- ERROR --: Error: HTTPSConnectionPool(host='130.237.35.90', port=49099): Max retries exceeded with url: /values/mapper/heater_mappings_bftc/device/c7/resistance/?prettyprint=1&key=1cfec235-b49c-4d3b-b7ec-f80ea46840a2 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x722f5cd00830>, 'Connection to 130.237.35.90 timed out. (connect timeout=3.0)'))
2026-02-02 14:09:56,381 :-- ERROR --: Error: HTTPSConnectionPool(host='130.237.35.90', port=49099): Max retries exceeded with url: /values/mapper/heater_mappings_bftc/device/c7/resistance/?prettyprint=1&key=1cfec235-b49c-4d3b-b7ec-f80ea46840a2 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x722f5cd00830>, 'Connection to 130.237.35.90 timed out. (connect timeout=3.0)'))
2026-02-02 14:09:56,381 :-- ERROR --: Error: HTTPSConnectionPool(host='130.237.35.90', port=49099): Max retries exceeded with url: /values/mapper/heater_mappings_bftc/device/c7/resistance/?prett

Channel 7 resistance: 0.0 Ohm


2026-02-02 14:09:59,413 :-- ERROR --: Error: HTTPSConnectionPool(host='130.237.35.90', port=49099): Max retries exceeded with url: /values/driver/bftc/data/heaters/heater_4/active/?prettyprint=1&key=1cfec235-b49c-4d3b-b7ec-f80ea46840a2 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x722f5cc04140>, 'Connection to 130.237.35.90 timed out. (connect timeout=3.0)'))
2026-02-02 14:09:59,413 :-- ERROR --: Error: HTTPSConnectionPool(host='130.237.35.90', port=49099): Max retries exceeded with url: /values/driver/bftc/data/heaters/heater_4/active/?prettyprint=1&key=1cfec235-b49c-4d3b-b7ec-f80ea46840a2 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x722f5cc04140>, 'Connection to 130.237.35.90 timed out. (connect timeout=3.0)'))
2026-02-02 14:09:59,413 :-- ERROR --: Error: HTTPSConnectionPool(host='130.237.35.90', port=49099): Max retries exceeded with url: /values/driver/bftc/data/heaters/heater_4/active/?prettyprint=1&key=1cfec235-b4

MXC heater status: False


KeyboardInterrupt: 